In [128]:
import re # We clean text using regex
import csv # To read the csv
from collections import defaultdict # For accumlating values
from gensim import corpora # To create corpus and dictionary for the LDA model
from gensim.models import LdaModel # To use the LDA model
import pandas as pd
import nltk

In [129]:
#Load reviews and classifier
fdsreviews_df=pd.read_csv("deliveroo.csv")
print("\nLoad reviews and classifier :")
print(fdsreviews_df[:40])

#Seperate Hashtags and titles to lists
review_list = fdsreviews_df["Customer_review"].tolist()


Load reviews and classifier :
             Customer_name                                       Customer_url  \
0                     Spud  https://www.productreview.com.au/consumer-prof...   
1                    Ariel  https://www.productreview.com.au/consumer-prof...   
2                     Tori  https://www.productreview.com.au/consumer-prof...   
3                      Roc  https://www.productreview.com.au/consumer-prof...   
4                    Claud  https://www.productreview.com.au/consumer-prof...   
5                  Shelley  https://www.productreview.com.au/consumer-prof...   
6                Yagnik J.  https://www.productreview.com.au/consumer-prof...   
7                      JAY  https://www.productreview.com.au/consumer-prof...   
8                      DDn  https://www.productreview.com.au/consumer-prof...   
9                      Dan  https://www.productreview.com.au/consumer-prof...   
10                 Jane S.  https://www.productreview.com.au/consumer-prof... 

In [130]:
data = pd.DataFrame(review_list)
data.head()

,0
0,Ensure that they only deliver to the front in ...
1,Didn’t receive my order. Agent reorders my foo...
2,Tried ordering online via the Deliveroo websit...
3,Live chat complaint was basically ignored orde...
4,Deliveroo will be the middleman when it comes ...


In [131]:
review_list = [re.sub(r'[^\w\s]','',str(item)) for item in review_list]

In [132]:
from nltk.corpus import stopwords # To remove stopwords
stopwords = set(stopwords.words('english'))

In [133]:
texts = [[word for word in document.lower().split() if word not in stopwords] for document in review_list]

In [134]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
         frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]

In [135]:
dictionary = corpora.Dictionary(texts)
print(dictionary)

Dictionary(1451 unique tokens: ['back', 'cold', 'customer', 'deliver', 'ensure']...)


In [136]:
corpus = [dictionary.doc2bow(text) for text in texts]
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)]]


In [137]:
NUM_TOPICS = 10 # This is an assumption. 
ldamodel = LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)#This might take some time.

In [124]:
topics = ldamodel.show_topics()
for topic in topics:
    print(topic)

(0, '0.018*"food" + 0.017*"order" + 0.016*"deliveroo" + 0.015*"never" + 0.014*"money" + 0.012*"get" + 0.011*"customer" + 0.011*"driver" + 0.010*"service" + 0.009*"would"')
(1, '0.039*"food" + 0.017*"never" + 0.017*"minutes" + 0.015*"got" + 0.015*"ordered" + 0.012*"deliveroo" + 0.012*"use" + 0.011*"hour" + 0.011*"service" + 0.010*"order"')
(2, '0.026*"service" + 0.021*"deliveroo" + 0.018*"order" + 0.017*"money" + 0.016*"meal" + 0.015*"customer" + 0.015*"never" + 0.015*"ordered" + 0.012*"told" + 0.010*"bank"')
(3, '0.047*"order" + 0.030*"deliveroo" + 0.024*"food" + 0.024*"restaurant" + 0.017*"would" + 0.017*"driver" + 0.015*"refund" + 0.012*"never" + 0.012*"said" + 0.011*"take"')
(4, '0.040*"deliveroo" + 0.021*"order" + 0.012*"ordered" + 0.011*"restaurant" + 0.010*"refund" + 0.009*"say" + 0.009*"like" + 0.009*"call" + 0.009*"told" + 0.008*"company"')
(5, '0.026*"order" + 0.022*"service" + 0.022*"food" + 0.020*"deliveroo" + 0.016*"delivery" + 0.016*"never" + 0.014*"time" + 0.011*"even" + 

In [125]:
word_dict = {};
for i in range(NUM_TOPICS):
    words = ldamodel.show_topic(i, topn = 40)
    word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]
pd.DataFrame(word_dict)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,food,food,service,order,deliveroo,order,address,food,food,order
1,order,never,deliveroo,deliveroo,order,service,order,deliveroo,service,food
2,deliveroo,minutes,order,food,ordered,food,deliveroo,order,delivery,time
3,never,got,money,restaurant,restaurant,deliveroo,got,service,call,deliveroo
4,money,ordered,meal,would,refund,delivery,driver,ordered,driver,hour
5,get,deliveroo,customer,driver,say,never,delivery,credit,order,driver
6,customer,use,never,refund,like,time,later,time,deliveroo,minutes
7,driver,hour,ordered,never,call,even,nothing,cold,never,service
8,service,service,told,said,told,rider,drivers,delivery,said,delivery
9,would,order,bank,take,company,restaurant,company,hour,would,restaurant


In [126]:
import pyLDAvis.gensim # To visualise LDA model effectively
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [127]:
# Compute Perplexity
print('\nPerplexity: ', ldamodel.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -6.700134442495201
